# eBay Web Scraping to identify features of Sponsored and Non-Sponsored items

In this project, I web-scraped both sponsored and non-sponsored "playstation 4 slim" items on eBay website. After obtaining the information of *"seller name, seller score, item price, # items sold, best offer available, title, returns allowed, shipping price, condition"*, I created a local database to store all these information. With some further statistical analysis using SQL, I identified some features of differentiating sponsored and non-sponsored items.

**Tasks to complete:**
* Study the eBay URL to find out terms for search, page number, number of items per page, buy it now.  

* Identify and differentiate SPONSORED/NON-SPONSORED items. I introduced two ways to do it. One is by regex, another is through HTML class.  

* **Put item URLs into two text files:** Loop through 10 pages, separately identify sponsored and non-sponsored items. Put their URLs into different text files.

* **Download item pages into two folders:** Open and read the two files containing item URLs. For each item URL, download the item page to the corresponding folder. Each item page should be named as "item_id.htm".  

* **Parse downloaded pages:** For each downloaded item page in the folders, parse them into Python project. 

* **Identify and store page elements:** Then identify the required HTML elements of the item page and store them. (seller name, seller score, item price, # items sold, best offer available, title, returns allowed, shipping price, condition)

* **Insert values into SQL database:** Create database and table. Import the previously stored info into the table.  

* **Stats analysis:** Use SQL in Python to run summary stats analysis on each item. 

1. The search query url link for "playstation 4 slim" is: https://www.ebay.com/sch/i.html?_nkw=playstation+4+slim  
The corresponding GET request's variable is "_nkw".   


2. The search query url link of 2nd search result page is: https://www.ebay.com/sch/i.html?_nkw=playstation+4+slim&_pgn=2  
The corresponding GET request's variable is "_pgn".   


3. The search query url link to include 100 items per search result page is: https://www.ebay.com/sch/i.html?_nkw=playstation+4+slim&_ipg=100  
The corresponding GET request's variable is "_ipg".   


4. The search query url link for only _buy-it-now_ items is: https://www.ebay.com/sch/i.html?_nkw=playstation+4+slim&rt=nc&LH_BIN=1  
The corresponding GET request's variable is "LH_BIN".   

In [1]:
import pandas as pd
import requests
import itertools
import json
from bs4 import BeautifulSoup as bs
from time import sleep
import os
import re
import pymysql
from tqdm import tqdm
from collections import OrderedDict

## Put URLs into two text files.
**Identify SPONSORED/NON-SPONSORED items for 10 pages. Store the URLs into two text files.**

In [5]:
# Constant definations. 
pages = [str(i) for i in range(1,11)]

# define Regex to find SPONSORED items and clean url.
spon_re = re.compile(r".*S.*P.*O.*N.*S.*O.*R.*E.*D.*")
link_re = re.compile(r"(.*)\?.*")

In [6]:
# For 10 pages 
s_links = []  # store all the sponsored items link.
ns_links = [] # store all the non-sponsored items link.
n_items =[]   # store the number of listings for each page.

agent = {"User-Agent":'Mozilla/5.0'}

for page in pages:
    
    # Get the response to different pages. Make soup.
    url = "https://www.ebay.com/sch/i.html?_nkw=playstation+4+slim&LH_BIN=1&rt=nc&_ipg=100&_pgn="+page
    response = requests.get(url,agent)
    soup = bs(response.text, "html.parser")

    # identify the listings.
    li = soup.findAll("li", class_="s-item")
    n_items.append(len(li)-1)

    # loop through the listings on the current page.
    for el in li:
        a = el.find("a")
        if(a==None): # not an actual item
            continue;

        span = el.find("span")
        text = span.text

        # identify SONSORED items using regex.
        if(spon_re.search(text)!=None):            # is a sponsored item.
            s_link = link_re.sub(r'\1',a["href"])  # make url clean using regex.
            s_links.append(s_link)
            # Write all the URLs of sponsored items' pages to the file "sponsored.txt"
            with open("sponsored.txt", "a") as file:
                file.write(s_link)
                file.write("\n")
        else:                                      # is a non-sponsored item.
            ns_link = link_re.sub(r'\1',a["href"]) # make url clean using regex.
            ns_links.append(ns_link) 
            # Write all the URLs of non-sponsored items' pages to the file "non-sponsored.txt"
            with open("non-sponsored.txt", "a") as file:
                file.write(ns_link)
                file.write("\n")
    sleep(2)
    print("Page "+page+" completed.")

Page 1 completed.
Page 2 completed.
Page 3 completed.
Page 4 completed.
Page 5 completed.
Page 6 completed.
Page 7 completed.
Page 8 completed.
Page 9 completed.
Page 10 completed.


### Another way to differentiate sponsored and non-sponsored items

The method used above is through Regex in "span" tag.   
Here's another method using "class" of "h3" tags.

By inspecting elements of eBay website, I found the *non-sponsored* items have "s-item__title" class for 'h3' tags.  
But sponsored items have "s-item__title s-item__title--has-tags" class for 'h3' tags.  
Therefore we used this class to differentiate.

But Note: <font color='red'>class='s-item__title s-item__title--has-tags'</font> does not mean the class stored is one string of 's-item__title s-item__title--has-tags'.  

In fact, the class is stored in list.  
<font color='red'>class='s-item__title s-item__title--has-tags'</font> means the class is stored as ['s-item__title', 's-item__title--has-tags'].

Therefore, if you use **soup.findAll('h3', class_='s-item__title s-item__title--has-tags')**, you can identify only sponsored items.  
But if you use **soup.findAll('h3', class_='s-item__title')**, you not only identified the non-sponsored items, but also sponsored items, namely all listings on the website.

To illustrate this point more clearly, let's program the below code:

In [4]:
# use the current soup of the final page as example.
h3_classes = []
h3 = soup.findAll('h3', class_='s-item__title')

for el in h3:  
    cl = (el['class'])
    h3_classes.append(cl)

# check the unique class values of the list

h3_classes.sort()
list(h3_classes for h3_classes,_ in itertools.groupby(h3_classes))


[['s-item__title'], ['s-item__title', 's-item__title--has-tags']]

In [ ]:
# differientiate non-sponsored and sponsored items

# identify h3 of all the listings.
h3 = soup.findAll('h3', class_='s-item__title')

# loop through the listings on the current page.
for el in h3:

    if("s-item__title--has-tags" in str(el["class"])):  # is a sponsored item.
        s_link = link_re.sub(r'\1',el.parent['href'])  # make url clean using regex.
        s_links.append(s_link)

    else:                                      # is a non-sponsored item.
        ns_link = link_re.sub(r'\1',el.parent['href']) # make url clean using regex.
        ns_links.append(ns_link) 

## Download item pages into two folders.
**Open and read the two files containing URLS. For each URL, download the page to the corresponding folder.**  
**Each file should be named as "item_id.htm".**

In [7]:
print("number of sponsored items:      "+ str( len(s_links))) 
print("number of non-sponsored items: "+ str(len(ns_links)))
print("total listings for 10 pages:   " + str(sum(n_items)))  

number of sponsored items:      123
number of non-sponsored items: 1001
total listings for 10 pages:   1124


In [8]:
# Create two folders
cwd=os.getcwd()
dir_ns = cwd+"/non-sponsored"
dir_s = cwd+"/sponsored"

os.mkdir(dir_s)
os.mkdir(dir_ns)

#### SPONSORED items:

In [9]:
# open and read file links.
os.chdir(cwd)

# define the Regex to indentify the itemid.
itemid_re = re.compile('.*(itm/[a-zA-Z0-9-]+/)(\d+)')
    
with open("sponsored.txt","r") as file:
    lines = file.readlines()
    file.close()

# store all the sponsored item id.
s_itemids = []
    
# for each url do loop.
os.chdir(dir_s)

for i in tqdm(range(0,len(lines))):
    url_line=lines[i]
    url_line = url_line.replace("\n","")
    # use Regex to find the itemid in the url.
    itemid = itemid_re.sub(r'\2',url_line)
    s_itemids.append(itemid)
    try:
        resp = requests.get(url_line,agent)
        sleep(2)
    except Exception:
        print("error occured here at: "+ str(i))
    # download pages.
    with open(itemid + ".htm", "w") as file:
        file.write(str(resp.text))

100%|██████████| 123/123 [05:28<00:00,  2.64s/it]


#### NON-SPONSORED items:

In [10]:
# open and read file links.
os.chdir(cwd)
with open("non-sponsored.txt","r") as file:
    lines = file.readlines()

# store all the non-sponsored item id.
ns_itemids = []

# for each url do loop.
os.chdir(dir_ns)

for i in tqdm(range(0,len(lines))):
    url_line=lines[i]
    url_line = url_line.replace("\n","")
    # use Regex to find the itemid in the url.
    itemid = itemid_re.sub(r'\2',url_line)
    ns_itemids.append(itemid)
    try:
        resp = requests.get(url_line,agent)
        sleep(2) 
    except Exception:
        print("error occured here: "+str(i))  
    # download pages.
    with open(itemid +".htm", "w") as file:
        file.write(str(resp.text))

100%|██████████| 1001/1001 [44:44<00:00,  2.65s/it]


#### Remove duplicates 
**(Not necessary as we catch the IntegrityError in the later codes.)**

In [12]:
print(len(s_itemids))
s_itemids = list(OrderedDict.fromkeys(s_itemids))
# s_itemids = list(set(s_itemids))
print(len(s_itemids))

print(len(ns_itemids))
ns_itemids = list(OrderedDict.fromkeys(ns_itemids))
# ns_itemids = list(set(ns_itemids))
print(len(ns_itemids))

itemids = ns_itemids+s_itemids
print(len(itemids))
itemids = list(OrderedDict.fromkeys(itemids))
print(len(itemids))

123
122
1001
1000
1122
1122


## Identify and store page elements.
**For each downloaded page in the folders, parse them into Python object.**   
**Then identify the required html elements and store them.**

Identify and select:   
seller name, seller score, item price, # items sold, best offer available, title, returns allowed, shipping price, condition.

In [13]:
selectors = ["sellername","sellerscore","itemprice","itemsold","betteroffer","title","returnallow","shippingprice","condit"]
# change the price to be "dollar-cent"format.
price_re = re.compile(r'.*\$([0-9,]+)\.(\d+).*')
# only extract the numbers in say "2,750 sold".
itemsold_re = re.compile(r'([0-9,]+).*')
# only extract the title but "Details about    "
title_re = re.compile(r'^(Details about)(\s+)(\w.*)')

For <font color=red>return allow</font> column, there are below unique values:  
- '14 day returns. Buyer pays for return shipping'  
- '30 days, buyer pays return shipping'  
- '30 day returns. Buyer pays for return shipping'  
- '60 day returns. Buyer pays for return shipping'  
- 'Free 30 day returns'  
- 'Free 60 day returns'  
- 'Seller does not accept returns'  

We transform this column to be <font color=red>binary</font>, with **'Seller does not accept returns'** to be 0, and **others** to be 1.

For <font color=red>shipping price</font> column, there are below unique values:    
- 'Varies based on location and shipping method'  
- 'Local pick-up offered.'  
- 'Free Local Pickup'  
- 'FREE'  
- 'FAST \'N FREE'  
- '999' and some other prices numbers.  

We define this column to be <font color=red>int</font>, with **'FAST \'N FREE', 'FREE'** to be 0.  
For **'Varies based on location and shipping method', 'Local pick-up offered.', 'Free Local Pickup'**, it basically means 'Varied' prices. As we cannot determine this, we put <font color=red>NULL</font> values.

#### SPONSORED items:

In [14]:
# loop through the SPONSORED downloaded pages.
os.chdir(dir_s)
s_soups=[]

# for each selector, we created the list to store all the info.
for selector in selectors:
    s_var = "s_"+selector+"s"
    exec("%s = []" %s_var) 

for i in tqdm(range(0,len(s_itemids))):
#for i in range(0,50):
    # Parse the current item page.
    with open(s_itemids[i]+".htm") as file: 
        s_soup = bs(file.read(),"html.parser")
        s_soups.append(s_soup) 
        file.close()
    
    try:
    # Selector commands:    
        # seller name element:
        sellername = s_soup.find("span", class_="mbg-nw").text
        s_sellernames.append(sellername)

        # seller score
        sellerscore = s_soup.find("span", class_="mbg-l").find("a").text
        s_sellerscores.append(int(sellerscore))

        # item price
        itemprice = s_soup.find("span", class_="notranslate").text
        # use Regex to convert the price into "dollar-cent" format.
        if("$" in str(itemprice)):
            itemprice = price_re.sub(r'\1\2',itemprice)
            itemprice = itemprice.replace(",","")
        # some prices are in other currencies, like GBP, we need to get the approximatly dollar price
        else: 
            itemprice = s_soup.find("span", id="convbinPrice").text
            itemprice = price_re.sub(r'\1\2',itemprice)
            itemprice = itemprice.replace(",","")
        s_itemprices.append(int(itemprice))

        # items sold
        itemsold = s_soup.find("a", class_="vi-txt-underline")
        if(itemsold!=None):
            itemsold=itemsold.text
            itemsold = itemsold_re.sub(r'\1',itemsold)
            itemsold = itemsold.replace(",","")
        else: 
            itemsold=None
        s_itemsolds.append(itemsold)

        # best offer avaiable
        bestoffer = s_soup.find("div",class_="vi-bbox-dspn u-flL lable boLable")
        if(bestoffer!= None):
            bestoffer = 1
        else:
            bestoffer = 0
        s_betteroffers.append(bestoffer)

        # title
        title = s_soup.find("h1", class_="it-ttl").text
        title = title_re.sub(r'\3',title)
        s_titles.append(title)

        # returns allowed
        returnallow = s_soup.find("span", id="vi-ret-accrd-txt").text
        if('Seller does not accept returns' in str(returnallow)):
            returnallow = 0
        else:
            returnallow = 1
        s_returnallows.append(returnallow)

        # shipping price
        shippingprice = s_soup.find("span", id="shSummary").find("span").text
        shippingprice = shippingprice.replace("\n","").replace("\t","")
        # use Regex to convert the price into "dollar-cent" format.
        if ("$" in str(shippingprice)):
            shippingprice = price_re.sub(r'\1\2',shippingprice)
        elif (('FAST' in str(shippingprice)) or ('FREE' == str(shippingprice))):
            shippingprice = 0
        else:
            shippingprice = None
        s_shippingprices.append(shippingprice)

        # condition
        condition = s_soup.find("div", id="vi-itm-cond").text
        s_condits.append(condition)
        
    except Exception:
        print("error occured here at: "+str(i))
        # Check whether there is error being caught.
        # s_sellernames.index("error_midterm")

100%|██████████| 122/122 [00:15<00:00,  7.86it/s]


#### NON-SPONSORED items:

In [28]:
# loop through the SPONSORED downloaded pages.
os.chdir(dir_ns)
ns_soups=[]

# for each selector, we created the list to store all the info.
for selector in selectors:
    ns_var = "ns_"+selector+"s"
    exec("%s = []" %ns_var) 

for i in tqdm(range(0,len(ns_itemids))):
#for i in range(0,50):
    # Parse the current item page.
    with open(ns_itemids[i]+".htm") as file: 
        ns_soup = bs(file.read(),"html.parser")
        ns_soups.append(ns_soup) 
        file.close()
    
    try:
    # Selector commands:    
        # seller name element:
        sellername = ns_soup.find("span", class_="mbg-nw").text
        ns_sellernames.append(sellername)

        # seller score
        sellerscore = ns_soup.find("span", class_="mbg-l").find("a").text
        ns_sellerscores.append(int(sellerscore))

        # item price
        itemprice = ns_soup.find("span", class_="notranslate").text
        # use Regex to convert the price into "dollar-cent" format.
        if("$" in str(itemprice)):
            itemprice = price_re.sub(r'\1\2',itemprice)
            itemprice = itemprice.replace(",","")
        # some prices are in other currencies, like GBP, we need to get the approximatly dollar price
        else:
            itemprice = s_soup.find("span", id="convbinPrice").text
            itemprice = price_re.sub(r'\1\2',itemprice)
            itemprice = itemprice.replace(",","")
        ns_itemprices.append(int(itemprice))

        # items sold
        itemsold = ns_soup.find("a", class_="vi-txt-underline")
        if(itemsold!=None):
            itemsold=itemsold.text
            itemsold = itemsold_re.sub(r'\1',itemsold)
            itemsold = itemsold.replace(",","")
        else: 
            itemsold=None
        ns_itemsolds.append(itemsold)

        # best offer avaiable
        bestoffer = ns_soup.find("div",class_="vi-bbox-dspn u-flL lable boLable")
        if(bestoffer!= None):
            bestoffer = 1
        else:
            bestoffer = 0
        ns_betteroffers.append(bestoffer)

        # title
        title = ns_soup.find("h1", class_="it-ttl").text
        title = title_re.sub(r'\3',title)
        ns_titles.append(title)

        # returns allowed
        returnallow = ns_soup.find("span", id="vi-ret-accrd-txt").text
        if('Seller does not accept returns' in str(returnallow)):
            returnallow = 0
        else:
            returnallow = 1
        ns_returnallows.append(returnallow)

        # shipping price
        shippingprice = ns_soup.find("span", id="shSummary")
        if(shippingprice!=None):
            shippingprice = shippingprice.find("span").text
            shippingprice = shippingprice.replace("\n","").replace("\t","")
            # use Regex to convert the price into "dollar-cent" format.
            if ("$" in str(shippingprice)):
                shippingprice = int(price_re.sub(r'\1\2',shippingprice))
            elif ( ('FAST' in str(shippingprice)) or ('FREE' == str(shippingprice))):
                shippingprice = 0
            else:
                shippingprice = None
        else:
            shippingprice = None
        ns_shippingprices.append(shippingprice)

        # condition
        condition = ns_soup.find("div", id="vi-itm-cond").text
        ns_condits.append(condition)
        
    except Exception:
        print("error occured here at: "+str(i))

100%|██████████| 1000/1000 [02:05<00:00,  7.98it/s]


## Insert values into SQL database.
**Create "eBay" database and "eBay_items" table.**
**Import the previously stored info into the table.**

Save the information of items in (d) into a single table named "eBay_items".    
This table should contain both sponsored and non-sponsored information and have a column that specifies which item is sponsored/non-sponsored.   
If an item misses ANY of the information in (d), you should insert that missing value as NULL into the table. 
Convert any price (item price and shipping price) into a "dollar-cent" format (e.g., convert "12.34" into 1234 and "12" into 1200). And insert the price as INT into the table.

In [29]:
pw = "" # a dynamic password for users to enter

In [30]:
# Create eBay Database and eBay_items table.
conn = pymysql.connect(host='localhost', user = 'root', password = pw)
cursor = conn.cursor()

# Create database eBay
SQL_DB = "eBay"
cursor.execute('DROP DATABASE IF EXISTS '+SQL_DB)
query_DB = "CREATE DATABASE IF NOT EXISTS " + SQL_DB + ";"
cursor.execute(query_DB)

# Create an empty table called eBay_items.
SQL_table = "eBay_items"
cursor.execute('DROP TABLE IF EXISTS ' + SQL_DB + '.' + SQL_table)
query_TB = "CREATE TABLE IF NOT EXISTS " + SQL_DB + "." + SQL_table + "(" \
+ "itemid VARCHAR(13) NOT NULL PRIMARY KEY" \
+ ", sellername VARCHAR(50)" \
+ ", sellerscore int(7)" \
+ ", itemprice int(6)" \
+ ", itemsold int(4)" \
+ ", betteroffer boolean" \
+ ", title VARCHAR(500)" \
+ ", returnallow boolean" \
+ ", shippingprice int(7)" \
+ ", condit VARCHAR(50)" \
+ ", sponsor_flag boolean" \
+ ");"

cursor.execute(query_TB);
cursor.close()
conn.close()

/anaconda3/lib/python3.7/site-packages/pymysql/cursors.py:170: Warning: (1051, "Unknown table 'ebay.ebay_items'")
  result = self._query(query)
/anaconda3/lib/python3.7/site-packages/pymysql/cursors.py:170: Warning: (1681, 'Integer display width is deprecated and will be removed in a future release.')
  result = self._query(query)


#### SPONSORED items:

In [31]:
# SPONSORED: Setup the SQL query
conn = pymysql.connect(host='localhost', user = 'root', password = pw, database=SQL_DB)
cursor = conn.cursor()

for i in tqdm(range(0,len(s_itemids))):
    sql = "INSERT INTO "+ SQL_table + ("(itemid,sellername,sellerscore,itemprice,itemsold,"
                                      "betteroffer,title,returnallow,shippingprice,condit,sponsor_flag)"
                              " VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, 1)")
    values = (s_itemids[i], s_sellernames[i], s_sellerscores[i], s_itemprices[i], s_itemsolds[i], \
              s_betteroffers[i], s_titles[i], s_returnallows[i], s_shippingprices[i],s_condits[i])
    
    # Execute the query
    try:
        cursor.execute(sql, values)
        conn.commit()
    except pymysql.IntegrityError:
        print(str(ns_itemids[i]) + ' already exists in the database.')

100%|██████████| 122/122 [00:00<00:00, 1645.35it/s]


#### NON-SPONSORED items:

In [32]:
# NON-SPONSORED: Setup the SQL query

for i in tqdm(range(0,len(ns_itemids))):
    sql = "INSERT INTO "+ SQL_table + ("(itemid,sellername,sellerscore,itemprice,itemsold,"
                                      "betteroffer,title,returnallow,shippingprice,condit,sponsor_flag)"
                              " VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, 0)")
    values = (ns_itemids[i], ns_sellernames[i], ns_sellerscores[i], ns_itemprices[i], ns_itemsolds[i], \
              ns_betteroffers[i], ns_titles[i], ns_returnallows[i], ns_shippingprices[i],ns_condits[i])
    
    # Execute the query
    try:
        cursor.execute(sql, values)
        conn.commit()
    except pymysql.IntegrityError:
        print(str(ns_itemids[i]) + ' already exists in the database.')

cursor.close()
conn.close()

100%|██████████| 1000/1000 [00:00<00:00, 2321.50it/s]


##  Run summary stats on each item

Use your code script (and NOT SQL GUI or command terminal) to run summary stats on each item. Print to the screen the **mean, min, max, and mean** for each column, grouped by "sponsor/non-sponsor" and "condition" (group by at the same time, not separately).   

If it is NOT a numerical/binary categorical column, print to the screen the count of each category level. 

You will need to ignore NULL values in your statistic calculations. 

#### Get SQL table as a python dataframs.

In [33]:
conn = pymysql.connect(host='localhost', user = 'root', password = pw, database=SQL_DB)

SQL_Query = pd.read_sql_query(
'''select
itemid,
sponsor_flag as s_flag,
sellerscore,
itemprice,
itemsold,
betteroffer,
returnallow,
shippingprice,
condit
from eBay_items''', conn)

eBay_items = pd.DataFrame(SQL_Query, columns=['itemid','s_flag','sellerscore','itemprice','itemsold',  \
                                              'betteroffer','returnallow','shippingprice','condit'])

conn.close()

eBay_items.head()

,itemid,s_flag,sellerscore,itemprice,itemsold,betteroffer,returnallow,shippingprice,condit
0,112619673851,0,258904,29900,1316.0,0,1,0.0,Seller refurbished
1,112780300205,0,15924,18999,93.0,0,1,0.0,Seller refurbished
2,113155080430,0,15924,22499,11.0,0,1,0.0,Seller refurbished
3,113354438187,1,258902,26900,58.0,0,1,0.0,Used
4,113751776516,0,3,30000,NaN,1,0,NaN,Used


#### Summary Stats

In [34]:
pd.options.display.float_format = "{:.2f}".format
pd.set_option('display.max_columns', 80)

In [39]:
eBay_items.groupby(['condit','s_flag']).aggregate(['count','mean','min','max','std']).reset_index()

condit s_flag sellerscore                         \
                                         count      mean min      max   
0  Manufacturer refurbished      0           5 453118.20   0  2265536   
1  Manufacturer refurbished      1           2    295.00  56      534   
2                       New      0         368  16214.50  -1   916855   
3                       New      1          47  13922.68   0   568380   
4                  Open box      0          63  22382.35   0   638526   
5                  Open box      1           4  16133.25   5    64483   
6        Seller refurbished      0          31 249748.39   0  2265543   
7        Seller refurbished      1           4  15097.75   0    41453   
8                      Used      0         533  52913.97   0   224444   
9                      Used      1          65  20898.88   0   258902   

             itemprice                                  itemsold              \
         std     count     mean    min     max      std    count   mean  min   
0 1013172.35         5 21359.80  19000   25000  2402.59        1   5.00 5.00   
1     338.00         2 26949.50  19999   33900  9829.49        1   3.00 3.00   
2   67467.40       368 32047.04  14300  126999 10912.55       86  77.53 1.00   
3   82762.15        47 33535.81  22100   59999  9995.55       19  19.53 1.00   
4  107617.62        63 26433.24  12500   52995  7418.23        5  28.60 1.00   
5   32233.17         4 27325.00  23800   33500  4548.53        1   5.00 5.00   
6  675945.45        31 23144.39  11200   36995  5429.10       10 155.30 1.00   
7   19605.55         4 22999.50  19999   25300  2462.25        2 871.50 4.00   
8   75571.99       533 22300.11   8500   50000  5134.23       10   5.60 1.00   
9   48012.05        65 28398.83  15000   55351  7509.41        3  29.00 8.00   

                  betteroffer                   returnallow                    \
      max     std       count mean min max  std       count mean min max  std   
0    5.00     nan           5 0.80   0   1 0.45           5 0.40   0   1 0.55   
1    3.00     nan           2 1.00   1   1 0.00           2 0.50   0   1 0.71   
2 1896.00  284.07         368 0.30   0   1 0.46         368 0.51   0   1 0.50   
3   99.00   27.96          47 0.32   0   1 0.47          47 0.66   0   1 0.48   
4  119.00   50.82          63 0.56   0   1 0.50          63 0.25   0   1 0.44   
5    5.00     nan           4 0.75   0   1 0.50           4 0.25   0   1 0.50   
6 1316.00  408.92          31 0.29   0   1 0.46          31 0.58   0   1 0.50   
7 1739.00 1226.83           4 0.25   0   1 0.50           4 0.75   0   1 0.50   
8   17.00    5.87         533 0.41   0   1 0.49         533 0.50   0   1 0.50   
9   58.00   25.94          65 0.49   0   1 0.50          65 0.54   0   1 0.50   

  shippingprice                               
          count    mean  min     max     std  
0             1    0.00 0.00    0.00     nan  
1             1    0.00 0.00    0.00     nan  
2           278  300.85 0.00 4854.00  837.10  
3            37  370.22 0.00 4999.00 1151.19  
4            20  324.90 0.00 2500.00  799.04  
5             1    0.00 0.00    0.00     nan  
6            23  290.30 0.00 2999.00  794.05  
7             3    0.00 0.00    0.00    0.00  
8           353  388.28 0.00 4660.00  812.53  
9            45 1062.53 0.00 5266.00 1816.66

In [36]:
# temp = eBay_items.groupby(['condit','s_flag']).describe()[["betteroffer","itemsold"]].reset_index()
# temp["betteroffer"]["count"]

**Look at Shipping Price**   
**Because 0 means FREE delivery, and None means varied shipping prices**

In [87]:
import numpy as np
# create a separate table to store Free and Varied shipping services items.
eBay_items_2 = eBay_items.loc[(eBay_items["shippingprice"]==0) | (eBay_items["shippingprice"].isnull()),]

In [88]:
temp1 = pd.DataFrame(eBay_items_2.groupby(['condit','s_flag'])["shippingprice"].value_counts())
temp2 = pd.DataFrame(eBay_items.groupby(['condit','s_flag'])["shippingprice"].count())

temp3 = pd.merge(temp1, temp2, on=['condit','s_flag']).reset_index()

In [89]:
temp3["free_ratio"] = temp3["shippingprice_x"]/temp3["shippingprice_y"]
temp3

,condit,s_flag,shippingprice_x,shippingprice_y,free_ratio
0,Manufacturer refurbished,0,1,1,1.00
1,Manufacturer refurbished,1,1,1,1.00
2,New,0,230,278,0.83
3,New,1,31,37,0.84
4,Open box,0,17,20,0.85
5,Open box,1,1,1,1.00
6,Seller refurbished,0,20,23,0.87
7,Seller refurbished,1,3,3,1.00
8,Used,0,275,353,0.78
9,Used,1,31,45,0.69


In [90]:
# look into the varied services proportion
# In the separate table (0 or None), we turn the Varied (None) into 1.
eBay_items_2.loc[eBay_items_2["shippingprice"].isnull(),"shippingprice"] = 1

# sum() is the number of items providing Varied shipping services.
temp4 = pd.DataFrame(eBay_items_2.groupby(['condit','s_flag'])["shippingprice"].sum())
temp5 = pd.DataFrame(eBay_items.groupby(['condit','s_flag'])["shippingprice"].count())

temp6 = pd.merge(temp4, temp5, on=['condit','s_flag']).reset_index()

/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [92]:
temp6["varied_ratio"] = temp6["shippingprice_x"]/(temp6["shippingprice_y"]+temp6["shippingprice_x"])
temp6

,condit,s_flag,shippingprice_x,shippingprice_y,varied_ratio
0,Manufacturer refurbished,0,4.00,1,0.80
1,Manufacturer refurbished,1,1.00,1,0.50
2,New,0,90.00,278,0.24
3,New,1,10.00,37,0.21
4,Open box,0,43.00,20,0.68
5,Open box,1,3.00,1,0.75
6,Seller refurbished,0,8.00,23,0.26
7,Seller refurbished,1,1.00,3,0.25
8,Used,0,180.00,353,0.34
9,Used,1,20.00,45,0.31


## Conclusions

Compared to NON-SPONSORED items (0):   

- SPONSORED items (1) have __higher__ average **seller scores**.  


- Except the "Seller Refurbished" condition, SPONSORED items (1) have __higher__ average **item prices**.  
- SPONSORED items (1) have __higher__ min **item prices**.  


- Except the "Seller Refurbished" condition, SPONSORED items (1) are __more likely__ to have **better offers**.   


- SPONSORED items (1) are __more likely__ to accept **return allow**.   


- Except for "Used" condition, SPONSORER items (1) are __more likely__ to offer **FREE Deliveries**.   
- Except for "Open Box" condition, SPONSORED items (1) are __less likely__ to offer **Varied shipping price Deliveries**. 


We can use higher seller scores, higher item prices, best offer available, retuan allow and Free shipping service to predict a SPONSORED item.